In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, roc_auc_score, confusion_matrix,
    precision_score, recall_score
)
import mlflow
import mlflow.sklearn
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
import os
from sklearn.model_selection import GridSearchCV


In [2]:

df1 = pd.read_csv("C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/data/one_hot_encode/train_processed_data.csv")



df2 = pd.read_csv("C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/data/one_hot_encode/test_processed_data.csv")


In [ ]:
df1

,num__person_age,num__person_income,num__person_emp_length,num__loan_amnt,num__loan_int_rate,num__loan_percent_income,num__cb_person_cred_hist_length,cat__person_home_ownership_OTHER,cat__person_home_ownership_OWN,cat__person_home_ownership_RENT,...,cat__loan_intent_PERSONAL,cat__loan_intent_VENTURE,cat__loan_grade_B,cat__loan_grade_C,cat__loan_grade_D,cat__loan_grade_E,cat__loan_grade_F,cat__loan_grade_G,cat__cb_person_default_on_file_Y,loan_status
0,1.397825,-0.366415,-1.182680,-0.359703,1.042717,-0.205762,2.274652,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
1,-0.091851,-0.542752,-0.429373,-0.755235,0.321836,-0.424086,0.787969,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
2,-0.422891,0.152225,-1.182680,-0.215873,-0.922425,-0.533248,-0.698714,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.894383,0.281885,-0.680475,1.042639,-1.363513,0.449211,1.283530,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,-0.919449,-0.480515,0.072832,-0.935022,-0.994843,-0.751573,-0.946495,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46911,2.887501,-0.107095,-0.931577,1.042639,-0.099501,0.995021,2.522433,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
46912,-0.919449,0.152225,-0.931577,0.143702,-1.337179,-0.205762,-0.698714,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
46913,0.735746,0.670864,-1.182680,-1.051884,-1.732183,-1.406545,0.044628,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
46914,-0.091851,1.189504,-0.680475,-1.114809,0.838632,-1.406545,0.787969,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0


In [3]:
xtrain=df1.drop(labels=["loan_status"],axis=1)
ytrain=df1[["loan_status"]]

In [4]:
xtest=df2.drop(labels=["loan_status"],axis=1)
ytest=df2[["loan_status"]]

In [ ]:
# Sampling the data

In [5]:
rus = RandomUnderSampler(sampling_strategy=1.0, random_state=42)  # 1.0 => equalize counts
X_res, y_res = rus.fit_resample(xtrain, ytrain)

In [8]:

log_reg = LogisticRegression(random_state=42)


param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga', 'lbfgs'],
    'max_iter': [100, 200, 500],
    'l1_ratio': [0, 0.5, 1]  
}


grid_search = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid,
    scoring='f1',       
    cv=5,               
    n_jobs=-1,          
    verbose=2
)

# Fit the grid search
grid_search.fit(X_res, y_res)

print("Best Parameters:",grid_search.best_params_)
print("Best Score:",grid_search.best_score_)

Fitting 5 folds for each of 756 candidates, totalling 3780 fits
Best Parameters: {'C': 100, 'l1_ratio': 0, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score: 0.8303848051791626


c:\ETLhive\python\venv\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
1890 fits failed out of a total of 3780.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
315 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ETLhive\python\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ETLhive\python\venv\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ETLhive\python\venv\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1218, in fit
    solver = _che

In [18]:
Best_Parameters = {'C': 100, 'l1_ratio': 0, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}

mlflow.set_tracking_uri("file:///C:/ML_AI/loan/loan_acceptance_using_multiple-algorithm/mlruns")
mlflow.set_experiment("loan_acceptance_logreg")

with mlflow.start_run(run_name=f"logreg_C={Best_Parameters['C']}_{Best_Parameters['penalty']}_{Best_Parameters['solver']}"):

    # log params (use Best_Parameters so variables are defined)
    mlflow.log_param("C", Best_Parameters["C"])
    mlflow.log_param("penalty", Best_Parameters["penalty"])
    mlflow.log_param("solver", Best_Parameters["solver"])
    mlflow.log_param("max_iter", Best_Parameters["max_iter"])
    mlflow.log_param("random_state", 42)

    model = LogisticRegression(
        C=Best_Parameters["C"],
        penalty=Best_Parameters["penalty"],
        solver=Best_Parameters["solver"],
        max_iter=Best_Parameters["max_iter"],
        random_state=42
    )
    model.fit(xtrain, ytrain)

    y_pred_train = model.predict(xtrain)
    y_prob_train = model.predict_proba(xtrain)[:, 1]

    acc_train = accuracy_score(ytrain, y_pred_train)
    roc_train = roc_auc_score(ytrain, y_prob_train)
    prec_train = precision_score(ytrain, y_pred_train)
    rec_train = recall_score(ytrain, y_pred_train)

    y_pred_test = model.predict(xtest)
    y_prob_test = model.predict_proba(xtest)[:, 1]

    acc_test = accuracy_score(ytest, y_pred_test)
    roc_test = roc_auc_score(ytest, y_prob_test)
    prec_test = precision_score(ytest, y_pred_test)
    rec_test = recall_score(ytest, y_pred_test)

    mlflow.log_metric("train_accuracy", acc_train)
    mlflow.log_metric("train_roc_auc", roc_train)
    mlflow.log_metric("train_precision", prec_train)
    mlflow.log_metric("train_recall", rec_train)

    mlflow.log_metric("test_accuracy", acc_test)
    mlflow.log_metric("test_roc_auc", roc_test)
    mlflow.log_metric("test_precision", prec_test)
    mlflow.log_metric("test_recall", rec_test)

    import os
    os.makedirs("metrics", exist_ok=True)

    cm_train = confusion_matrix(ytrain, y_pred_train)
    cm_test = confusion_matrix(ytest, y_pred_test)

    np.save("metrics/confusion_matrix_train.npy", cm_train)
    np.save("metrics/confusion_matrix_test.npy", cm_test)

    mlflow.log_artifact("metrics/confusion_matrix_train.npy", artifact_path="metrics")
    mlflow.log_artifact("metrics/confusion_matrix_test.npy", artifact_path="metrics")

    mlflow.sklearn.log_model(model, artifact_path="model")

    print(f"Run logged: C={Best_Parameters['C']}, penalty={Best_Parameters['penalty']}, solver={Best_Parameters['solver']}")
    print(f"Train: Acc={acc_train:.3f}, ROC={roc_train:.3f}, Prec={prec_train:.3f}, Rec={rec_train:.3f}")
    print(f"Test : Acc={acc_test:.3f}, ROC={roc_test:.3f}, Prec={prec_test:.3f}, Rec={rec_test:.3f}")


c:\ETLhive\python\venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
2025/11/12 14:37:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 14:37:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run logged: C=100, penalty=l1, solver=liblinear
Train: Acc=0.913, ROC=0.901, Prec=0.781, Rec=0.540
Test : Acc=0.912, ROC=0.905, Prec=0.773, Rec=0.526


In [ ]:
# Your best parameters from GridSearch
Best_Parameters = {'C': 100, 'l1_ratio': 0, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}


with mlflow.start_run(run_name=f"logreg_C={Best_Parameters['C']}_{Best_Parameters['penalty']}_{Best_Parameters['solver']}"):

 
    mlflow.log_param("C", Best_Parameters["C"])
    mlflow.log_param("penalty", Best_Parameters["penalty"])
    mlflow.log_param("solver", Best_Parameters["solver"])
    mlflow.log_param("max_iter", Best_Parameters["max_iter"])
    mlflow.log_param("random_state", 42)

   
    model = LogisticRegression(
        C=Best_Parameters["C"],
        penalty=Best_Parameters["penalty"],
        solver=Best_Parameters["solver"],
        max_iter=Best_Parameters["max_iter"],
        random_state=42
    )
    model.fit(xtrain, ytrain)


    y_pred_train = model.predict(xtrain)
    y_prob_train = model.predict_proba(xtrain)[:, 1]

    acc_train = accuracy_score(ytrain, y_pred_train)
    roc_train = roc_auc_score(ytrain, y_prob_train)
    prec_train = precision_score(ytrain, y_pred_train)
    rec_train = recall_score(ytrain, y_pred_train)

   
    y_pred_test = model.predict(xtest)
    y_prob_test = model.predict_proba(xtest)[:, 1]

    acc_test = accuracy_score(ytest, y_pred_test)
    roc_test = roc_auc_score(ytest, y_prob_test)
    prec_test = precision_score(ytest, y_pred_test)
    rec_test = recall_score(ytest, y_pred_test)

  
    mlflow.log_metric("train_accuracy", acc_train)
    mlflow.log_metric("train_roc_auc", roc_train)
    mlflow.log_metric("train_precision", prec_train)
    mlflow.log_metric("train_recall", rec_train)

    mlflow.log_metric("test_accuracy", acc_test)
    mlflow.log_metric("test_roc_auc", roc_test)
    mlflow.log_metric("test_precision", prec_test)
    mlflow.log_metric("test_recall", rec_test)

    
    os.makedirs("metrics", exist_ok=True)

    cm_train = confusion_matrix(ytrain, y_pred_train)
    cm_test = confusion_matrix(ytest, y_pred_test)

    np.save("metrics/confusion_matrix_train.npy", cm_train)
    np.save("metrics/confusion_matrix_test.npy", cm_test)

    mlflow.log_artifact("metrics/confusion_matrix_train.npy", artifact_path="metrics")
    mlflow.log_artifact("metrics/confusion_matrix_test.npy", artifact_path="metrics")

    
    mlflow.sklearn.log_model(model, artifact_path="model")

    print(f"Run logged: C={Best_Parameters['C']}, penalty={Best_Parameters['penalty']}, solver={Best_Parameters['solver']}")
    print(f"Train: Acc={acc_train:.3f}, ROC={roc_train:.3f}, Prec={prec_train:.3f}, Rec={rec_train:.3f}")
    print(f"Test : Acc={acc_test:.3f}, ROC={roc_test:.3f}, Prec={prec_test:.3f}, Rec={rec_test:.3f}")


c:\ETLhive\python\venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
2025/11/12 14:26:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 14:27:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run logged: C=100, penalty=l1, solver=liblinear
Train: Acc=0.913, ROC=0.901, Prec=0.781, Rec=0.540
Test : Acc=0.912, ROC=0.905, Prec=0.773, Rec=0.526
